In [16]:
import pandas as pd
import requests

####  1) Lendo de APIs
Carregamento de dados públicos do governo através de um API (*Application Programming Interface*). Dados de pedidos de verificação de limites (PVL) realizados por estados ou município.

1. Seleção de PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. 
4. Indicando a frequência de cada ano do campo construído no item (3).

In [17]:
# 1) Seu código aqui
link = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
df = requests.get(link)
df_d = df.json()

In [18]:
df_rj = pd.DataFrame(df_d['items'])
df_rj.dtypes

id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [19]:
df_rj

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,8528,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.000793/2009-56,2009-07-17T03:00:00Z,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.570000e+08,1,0,24/07/2009
1,7057,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000487/2014-87,2014-04-24T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.000000e+09,1,0,04/07/2014
2,7084,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000493/2010-19,2010-08-06T03:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.867300e+07,1,0,19/08/2010
3,22594,Estado,Rio de Janeiro,33,RJ,00000.000000/2031-89,Arquivado por decurso de prazo,17944.000657/2016-95,2016-05-12T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,9.892104e+08,0,0,25/10/2016
4,10606,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001240/2009-11,2009-10-22T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,2.000000e+07,1,0,04/11/2009
5,12410,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001680/2011-92,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,6.000000e+08,1,0,10/07/2012
6,9050,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000893/2010-16,2011-06-03T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.591383e+08,1,0,04/07/2011
7,6660,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.000403/2010-81,2010-05-24T03:00:00Z,Operação contratual interna,PAC - Favelas,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.060466e+08,1,0,06/07/2010
8,13271,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001999/2008-12,2009-07-01T03:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.950000e+07,1,0,07/07/2009
9,5933,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000237/2012-85,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,1.000000e+08,1,0,10/07/2012


In [20]:
# 2) Seu código aqui
df_rj['status'].value_counts().head(3)

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: status, dtype: int64

In [21]:
# 3) Seu código aqui
from datetime import datetime
df_rj['data_status'] = pd.to_datetime(df_rj['data_status'])
df_rj['year'] = pd.DatetimeIndex(df_rj['data_status']).year

In [22]:
df_rj['year'].value_counts()

2012    11
2009     9
2013     7
2011     6
2014     4
2016     4
2002     4
2010     3
2008     2
2006     2
Name: year, dtype: int64

####  2) Melhorando a interação com o API
Criada função para consultar Api com Estado e tipo de interesse variável.

1. Função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [23]:
#1) Seu código aqui
def consulta_api(uf, estado_ou_municipio):
    uf = uf.upper()
    estado_ou_municipio = estado_ou_municipio.title()
    link = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={estado_ou_municipio}'
    df = requests.get(link)
    df = df.json()
    df = pd.DataFrame(df['items'])
    return df

In [24]:
# 2) Seu código aqui
df_new_mg = consulta_api('mg', 'Estado')
df_new_mg

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7032,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000482/2012-92,2012-10-11T03:00:00Z,Operação contratual externa (com garantia da U...,Reestruturação e recomposição do principal de ...,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,4.500000e+08,1,0,15/10/2012
1,11328,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001412/2012-51,2012-12-11T02:00:00Z,Operação contratual interna (com garantia da U...,Multissetorial,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.653733e+09,1,0,03/01/2013
2,13618,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000037/2006-15,2009-02-26T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/03/2009
3,7280,Estado,Minas Gerais,31,MG,None,Deferido,17944.000537/2010-01,2010-06-24T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.488900e+07,1,0,25/06/2010
4,4926,Estado,Minas Gerais,31,MG,None,Arquivado,10951.000367/1998-24,1998-05-12T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.650000e+08,0,0,24/07/2002
5,10338,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001175/2009-23,2009-09-22T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.781800e+08,1,0,08/10/2009
6,11054,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001346/2012-10,2012-10-29T02:00:00Z,Operação contratual interna (com garantia da U...,Proinveste,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.326390e+09,1,0,02/01/2013
7,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,29/04/2016
8,13447,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000002/2006-78,2009-02-02T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/11/2009
9,7002,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000477/2010-18,2010-05-24T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.672700e+08,1,0,17/06/2010


In [25]:
# 3) Seu código aqui
df_new_adp = df_new_mg['status'].value_counts()
qtd_arq_dec_prz = df_new_mg[df_new_mg['status'] == 'Arquivado por decurso de prazo']
df_new_mg

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7032,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000482/2012-92,2012-10-11T03:00:00Z,Operação contratual externa (com garantia da U...,Reestruturação e recomposição do principal de ...,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,4.500000e+08,1,0,15/10/2012
1,11328,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001412/2012-51,2012-12-11T02:00:00Z,Operação contratual interna (com garantia da U...,Multissetorial,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.653733e+09,1,0,03/01/2013
2,13618,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000037/2006-15,2009-02-26T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/03/2009
3,7280,Estado,Minas Gerais,31,MG,None,Deferido,17944.000537/2010-01,2010-06-24T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.488900e+07,1,0,25/06/2010
4,4926,Estado,Minas Gerais,31,MG,None,Arquivado,10951.000367/1998-24,1998-05-12T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.650000e+08,0,0,24/07/2002
5,10338,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001175/2009-23,2009-09-22T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.781800e+08,1,0,08/10/2009
6,11054,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001346/2012-10,2012-10-29T02:00:00Z,Operação contratual interna (com garantia da U...,Proinveste,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.326390e+09,1,0,02/01/2013
7,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,29/04/2016
8,13447,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000002/2006-78,2009-02-02T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/11/2009
9,7002,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000477/2010-18,2010-05-24T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.672700e+08,1,0,17/06/2010


In [26]:
# 3) Seu código aqui
df_new_adp = df_new_mg['status'].value_counts()
qtd_arq_dec_prz = df_new_mg[df_new_mg['status'] == 'Arquivado por decurso de prazo']
df_new_mg

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7032,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000482/2012-92,2012-10-11T03:00:00Z,Operação contratual externa (com garantia da U...,Reestruturação e recomposição do principal de ...,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,4.500000e+08,1,0,15/10/2012
1,11328,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001412/2012-51,2012-12-11T02:00:00Z,Operação contratual interna (com garantia da U...,Multissetorial,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.653733e+09,1,0,03/01/2013
2,13618,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000037/2006-15,2009-02-26T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/03/2009
3,7280,Estado,Minas Gerais,31,MG,None,Deferido,17944.000537/2010-01,2010-06-24T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.488900e+07,1,0,25/06/2010
4,4926,Estado,Minas Gerais,31,MG,None,Arquivado,10951.000367/1998-24,1998-05-12T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.650000e+08,0,0,24/07/2002
5,10338,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001175/2009-23,2009-09-22T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.781800e+08,1,0,08/10/2009
6,11054,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001346/2012-10,2012-10-29T02:00:00Z,Operação contratual interna (com garantia da U...,Proinveste,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.326390e+09,1,0,02/01/2013
7,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,29/04/2016
8,13447,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000002/2006-78,2009-02-02T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/11/2009
9,7002,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000477/2010-18,2010-05-24T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.672700e+08,1,0,17/06/2010


In [27]:
# 4) Seu código aqui
df_new_ba = consulta_api('ba', 'município')
df_new_ba

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,8401,Município,Araci,2902104,BA,None,Arquivado por decurso de prazo,17944.000770/2012-47,2012-08-15T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2184375.16,0,0,20/08/2013
1,9216,Município,Igrapiúna,2913457,BA,None,Pendente de regularização,17944.000929/2012-23,2016-01-11T02:00:00Z,Operação contratual interna,Regularização de Dívida - Água e Esgoto,Empresa Estatal,Empresa Baiana de Águas e Saneamento S/A,Real,94971.70,0,0,13/10/2016
2,10453,Município,Serrinha,2930501,BA,None,Regularizado,17944.001203/2012-16,2013-07-12T03:00:00Z,Operação contratual interna,Regularização de Dívida - Água e Esgoto,Empresa Estatal,Empresa Baiana de Águas e Saneamento S/A,Real,329342.05,0,0,14/08/2013
3,10462,Município,Mirangaba,2921401,BA,None,Arquivado por decurso de prazo,17944.001205/2013-88,2013-11-21T02:00:00Z,Operação contratual interna,Provias 2012,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1250000.00,0,0,15/09/2014
4,10014,Município,Aramari,2902203,BA,None,Arquivado,17944.001100/2008-61,2008-04-07T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,299450.00,0,0,13/08/2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,32153,Município,Cairu,2905404,BA,PVL02.001315/2019-72,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,4480000.00,0,0,22/05/2019
848,27466,Município,Euclides da Cunha,2910701,BA,PVL02.000085/2018-43,Arquivado por decurso de prazo,17944.100602/2018-46,2018-01-16T21:16:12Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15000000.00,0,0,16/08/2018
849,27328,Município,Capim Grosso,2906873,BA,PVL02.002663/2017-03,Arquivado por decurso de prazo,17944.100217/2018-07,2017-12-29T14:41:32Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4968418.64,0,0,13/08/2018
850,6899,Município,Ipecaetá,2913804,BA,None,Deferido,17944.000455/2011-39,2011-05-03T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,180000.00,0,0,19/05/2011


In [28]:
df_new_ba_status = df_new_ba[df_new_ba['status'] == 'Deferido']
df_new_ba_status = df_new_ba[['interessado','status']]
df_new_ba_status.value_counts().head(1)

interessado             status  
Luís Eduardo Magalhães  Deferido    14
dtype: int64

In [29]:
# 5) Seu código aqui
df_new_ba = consulta_api('ba', 'estado')
df_new_ba.to_csv('./dados_solicitacao_ba.csv')